# Tutorial: Creating TF Records

Last updated: 09-sep-2021 by
Cristobal Donoso

In [1]:
cd ../..

/home/users/cdonoso/astromer/ASTROMER


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import sys,os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from core.masking import get_padding_mask
from core.astromer import get_ASTROMER, train
from core.data  import (create_dataset, 
                        pretraining_records)

# This tutorial runs only in early mode. For the optimized code use train.py
# OTHERWISE you can comment the next ljupyine and reset the kernel before the finetuning
tf.config.run_functions_eagerly(False)

%load_ext autoreload
%autoreload 2

In [12]:
source = './data/raw_data/atlas/LCs/filter_o/' # lightcurves folder
metadata = './data/raw_data/atlas/metadata_o.dat' # metadata file
name = 'big_atlas'

### Metadata frame sample

In [16]:
meta = pd.read_csv(metadata)
print(meta.shape)
# meta = meta[meta['Class'] != 'UNK']
# meta = meta[meta['Class'] != 'std']
# meta = meta[meta['Class'] != 'Dubious']
# meta = meta[meta['N'] >= 20]
meta['Band'] = tf.ones(meta.shape[0])
meta = meta.rename(columns={'objID':'ID', 'Unnamed: 0':'ID', 'Path_R':'Path'})
print(meta.shape)
if 'atlas' in name:
    meta['Path'] = meta['ID'].astype(str)+'.dat'

(4719921, 4)
(4719921, 5)


In [17]:
# meta.sample()

### Lightcurve frame sample

In [22]:
lc_df = pd.read_csv(os.path.join(source,  
                                 meta['Path'].sample(1).values[0].split('/')[-1]), 
                    sep=';')
lc_df.head()

,Time,Mag,Mag_err
0,57228.37494,15.302,0.020
1,57228.40180,15.272,0.018
2,57228.42744,15.274,0.019
3,57228.45248,15.301,0.017
4,57288.25446,15.172,0.035


### Creating training partitions from .csv to .record 

In [23]:
meta.shape

(4719921, 6)

In [13]:
%%time

for fold_n in range(3): 
    test_meta  = pd.concat([frame.sample(n=100) for g, frame in meta.groupby('Class')])
    train_meta = meta[~meta['ID'].isin(test_meta['ID'])]
    print(test_meta.shape, train_meta.shape)
        
    for nsamples in [0]:  
        if nsamples == 0:
            partial_meta = train_meta
            target = './data/records/{}/fold_{}/{}'.format(name, fold_n, name)
        else:
            partial_meta = pd.concat([frame.sample(n=nsamples) for c, frame in train_meta.groupby('Class')])
            target = './data/records/{}/fold_{}/{}_{}'.format(name, fold_n, name, nsamples)

        create_dataset(partial_meta, source, target, max_lcs_per_record=20000, 
                       n_jobs=7, subsets_frac=(0.8, 0.2), test_subset=test_meta,  
                       names=['mjd', 'mag', 'errmag'],
                       delim_whitespace=True)

(1000, 5) (357288, 5)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:39<00:00, 39.94s/it]


(1000, 5) (357288, 5)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:37<00:00, 39.75s/it]


(1000, 5) (357288, 5)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:37<00:00, 39.78s/it]

CPU times: user 16min 54s, sys: 20.2 s, total: 17min 15s
Wall time: 19min 55s


### JUST ONE DATASET

In [24]:
name

'big_atlas'

In [8]:
for fold_n in range(1):
    target = './data/records/{}/fold_{}/{}'.format(name, fold_n, name)
    
    test_meta  = meta.sample(frac=0.2)
    train_meta = meta[~meta['ID'].isin(test_meta['ID'])]

    create_dataset(train_meta, source, target, max_lcs_per_record=20000, 
                   n_jobs=7, subsets_frac=(0.8, 0.2), test_subset=test_meta)

100%|██████████████████████████████████████████████████████████████████| 11/11 [01:17<00:00,  7.04s/it]


In [17]:
root = './data/records/new_ogle/'
for fold_n in range(3):
    fold_f = os.path.join(root, 'fold_{}'.format(fold_n))
    for source in os.listdir(fold_f):
        target = source.replace('new_', '')
        os.rename(os.path.join(fold_f, source), os.path.join(fold_f, target))